# Système RAG avec Routage Intelligent

Ce notebook implémente un système RAG (Retrieval-Augmented Generation) avec plusieurs stratégies de routage pour améliorer la qualité des réponses.

## Routage par Type de Requête

Ce type de routage analyse la nature de la question de l'utilisateur pour choisir une chaîne de traitement adaptée.

### Simple vs. Complexe 

**Mécanisme :** Un LLM classifie la question. 
- **Simple** : Répondable directement avec une seule requête sur les données. Utilise un RAG direct.
- **Complexe** : Question à plusieurs étapes, plusieurs concepts ou comparaison. Le système décompose la question en sous-questions, les traite individuellement, puis synthétise une réponse finale.

### Configuration initiale

Cette cellule configure le LLM (Ollama/Mistral) et charge le vector store.

In [ ]:
# ============================================================================
# Imports et Configuration Initiale
# ============================================================================

from langchain_community.chat_models import ChatOllama
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Configuration du LLM (Ollama avec modèle Mistral)
# Assurez-vous qu'Ollama est installé et que le modèle mistral est téléchargé
# Installation : https://ollama.ai/
llm = ChatOllama(model="mistral")

# Modèle d'embedding (doit correspondre EXACTEMENT à celui utilisé lors de la création des vector stores)
# IMPORTANT: Utilisez le même modèle que dans rag_embedding_storing.ipynb
# Par défaut: BAAI/bge-large-en-v1.5 (ou BAAI/bge-base-en-v1.5 pour une version plus légère)
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

# Charger le vector store (assurez-vous que les vector stores ont été créés avec rag_embedding_storing.ipynb)
# Pour un vector store simple (sans séparation import/export)
# Note: Si vous utilisez des vector stores séparés (import/export), voir la section suivante
retriever = Chroma(
    persist_directory="chroma_store", 
    embedding_function=embedding_model
).as_retriever()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Implémentation du routage Simple/Complexe

Cette section contient :
- Les templates de prompts
- Le classificateur de type de question
- Le pipeline de décomposition pour questions complexes
- La fonction principale de routage

In [ ]:
# ============================================================================
# Imports pour le Routage Intelligent
# ============================================================================

from typing import Literal
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ============================================================================
# Templates de Prompts
# ============================================================================

# Template principal pour les réponses RAG
template = """You are a helpful assistant.

Conversation history:
{history}

Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# ============================================================================
# 1. Classificateur Simple vs Complexe
# ============================================================================

detection_prompt = ChatPromptTemplate.from_template(
    """
Tu es un assistant expert. Ta tâche est de déterminer si une question est simple ou complexe.
- Simple : répondable directement avec une seule requête sur les données.
- Complexe : question à plusieurs étapes, plusieurs concepts ou comparaison.

Question : {question}
Catégorie (simple ou complexe) :
"""
)

def detect_query_type(question: str) -> Literal["simple", "complexe"]:
    """
    Détermine si une question est simple ou complexe.
    
    Args:
        question: La question de l'utilisateur
        
    Returns:
        "simple" ou "complexe"
    """
    output = (detection_prompt | llm | StrOutputParser()).invoke({"question": question})
    if "complexe" in output.lower():
        return "complexe"
    return "simple"

# ============================================================================
# 2. Pipeline de Décomposition pour Questions Complexes
# ============================================================================

prompt_decomposition = ChatPromptTemplate.from_template(
    """
You are a helpful assistant that generates multiple sub-questions related to an input question.
The goal is to break down the input into a set of sub-problems / sub-questions that can be answered in isolation.
Generate multiple search queries related to: {question}
Output (3 queries):
"""
)

# Chaîne pour générer les sous-questions
generate_subqs = (
    prompt_decomposition 
    | llm 
    | StrOutputParser() 
    | (lambda x: x.strip().split("\n"))
)

# Template pour la fusion des réponses aux sous-questions
fusion_template = ChatPromptTemplate.from_template(
    """
Conversation history:
{history}

Voici les réponses à chaque sous-question :
{details}

Répond maintenant à la question initiale :
{question}
"""
)

def format_history(history):
    """Formate l'historique de conversation en texte."""
    return "\n".join(f"{msg.type}: {msg.content}" for msg in history)

# ============================================================================
# 3. Pipeline de Routage Intelligent
# ============================================================================

def hybrid_routing_pipeline(query: str):
    """
    Pipeline principal qui route les questions vers le traitement approprié.
    
    - Questions simples : RAG direct
    - Questions complexes : Décomposition en sous-questions puis synthèse
    
    Args:
        query: La question de l'utilisateur
        
    Returns:
        La réponse générée
    """
    # Détecter le type de question
    query_type = detect_query_type(query)
    print(f"Type de question détecté : {query_type}")

    # Chaîne RAG de base
    runnable_chain = prompt | llm | StrOutputParser()

    # Traitement des questions simples
    if query_type == "simple":
        docs = retriever.get_relevant_documents(query)
        context = "\n\n".join(doc.page_content for doc in docs)
        return runnable_chain.invoke({
            "context": context, 
            "question": query,
            "history": ""
        })

    # Traitement des questions complexes : décomposition
    chat_history = InMemoryChatMessageHistory()
    chat_history.add_user_message(query)
    
    # Générer les sous-questions
    subqs = generate_subqs.invoke({"question": query})
    print(f"Sous-questions générées : {len(subqs)}")

    # Répondre à chaque sous-question
    answers = []
    for subq in subqs:
        if not subq.strip():
            continue
        chat_history.add_user_message(subq)
        docs = retriever.get_relevant_documents(subq)
        context = "\n\n".join(doc.page_content for doc in docs)
        resp = runnable_chain.invoke({
            "context": context, 
            "question": subq,
            "history": ""
        })
        chat_history.add_ai_message(resp)
        answers.append((subq, resp))

    # Fusionner les réponses
    details = "\n\n".join(f"Q: {q}\nA: {a}" for q, a in answers)
    history_text = format_history(chat_history.messages)

    final_chain = fusion_template | llm | StrOutputParser()
    final_answer = final_chain.invoke({
        "details": details,
        "question": query,
        "history": history_text
    })

    chat_history.add_ai_message(final_answer)
    return final_answer

# ============================================================================
# Exemple d'utilisation
# ============================================================================

if __name__ == "__main__":
    question = "Quelles sont les exigences et les documents nécessaires pour importer du matériel médical temporairement au Maroc ?"
    result = hybrid_routing_pipeline(question)
    print("\n" + "="*80)
    print("RÉPONSE FINALE:")
    print("="*80)
    print(result)


### Configuration des vector stores Import/Export

Charge les vector stores séparés pour l'import et l'export créés dans `rag_embedding_storing.ipynb`.

## Routage vers Import/Export Vector Store

Cette section implémente un routage qui dirige les questions vers le bon vector store selon qu'elles concernent l'import ou l'export.

**Mécanisme :** Un classificateur LLM détermine si la question concerne l'import ou l'export, puis route la requête vers le vector store approprié.

### Implémentation du routage Import/Export

Cette section contient :
- Le classificateur Import/Export
- La fonction de routage vers le bon vector store

In [ ]:
# ============================================================================
# Configuration des Vector Stores Import/Export
# ============================================================================
# Note: Assurez-vous d'avoir créé les vector stores avec rag_embedding_storing.ipynb
# avant d'exécuter cette cellule

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Modèle d'embedding (doit correspondre à celui utilisé lors de la création)
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

# Charger les vector stores persistés
vs_import = Chroma(
    persist_directory="chroma_store/import",
    embedding_function=embedding_model,
    collection_name="import_docs"
)

vs_export = Chroma(
    persist_directory="chroma_store/export",
    embedding_function=embedding_model,
    collection_name="export_docs"
)

# Créer les retrieveurs
retriever_import = vs_import.as_retriever()
retriever_export = vs_export.as_retriever()

print("✅ Vector stores import/export chargés avec succès")

# ============================================================================
# Classificateur Import/Export
# ============================================================================

classification_prompt = ChatPromptTemplate.from_template("""
Tu es un assistant qui doit déterminer si une question concerne l'import ou l'export.

Catégorise cette question : "{question}"

Réponds uniquement par : import ou export.
""")

def detect_import_or_export(question: str) -> str:
    """
    Détermine si une question concerne l'import ou l'export.
    
    Args:
        question: La question de l'utilisateur
        
    Returns:
        "import" ou "export"
    """
    output = (classification_prompt | llm | StrOutputParser()).invoke({"question": question})
    return "import" if "import" in output.lower() else "export"

# ============================================================================
# Router Import/Export
# ============================================================================

def import_export_router(question: str):
    """
    Route une question vers le vector store approprié (import ou export).
    
    Args:
        question: La question de l'utilisateur
        
    Returns:
        La réponse générée à partir du bon vector store
    """
    # Détecter le type de flux
    target = detect_import_or_export(question)
    print(f"Type de flux détecté : {target}")

    # Sélectionner le bon retriever
    retriever = retriever_import if target == "import" else retriever_export

    # Récupérer les documents pertinents
    docs = retriever.get_relevant_documents(question)
    context = "\n\n".join(doc.page_content for doc in docs)

    # Template de prompt pour la réponse
    rag_prompt = ChatPromptTemplate.from_template(
        """Réponds à la question suivante à partir des informations ci-dessous.

CONTEXTE:
{context}

QUESTION:
{question}"""
    )

    # Générer la réponse
    answer = (
        {"context": context, "question": question} 
        | rag_prompt 
        | llm 
        | StrOutputParser()
    ).invoke()
    
    return answer

# ============================================================================
# Exemple d'utilisation
# ============================================================================

if __name__ == "__main__":
    question = "Quels sont les documents nécessaires pour exporter du textile au Maroc ?"
    result = import_export_router(question)
    print("\n" + "="*80)
    print("RÉPONSE:")
    print("="*80)
    print(result)
